In [1]:
# Import necessary modules
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
%matplotlib inline

In [2]:
# Homes contains data on home sales, zipcodes is for encoding cities later
homes = pd.read_csv('data/kc_house_data.csv')
zipcodes = pd.read_csv('data/zips.csv')

# Columns suggested to drop by project description
cols_to_drop = (['id','date','view','sqft_above','sqft_basement','yr_renovated',
                'lat','long','sqft_living15','sqft_lot15'])
homes = homes.drop(cols_to_drop,axis=1)

# Exchange the zipcode column for a city column based on zipcode
homes = homes.merge(zipcodes,how='left',on='zipcode').drop('zipcode',axis=1)
homes.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,condition,grade,yr_built,city
0,221900.0,3,1.00,1180,5650,1.0,NaN,Average,7 Average,1955,Seattle
1,538000.0,3,2.25,2570,7242,2.0,NO,Average,7 Average,1951,Seattle
2,180000.0,2,1.00,770,10000,1.0,NO,Average,6 Low Average,1933,Kenmore
3,604000.0,4,3.00,1960,5000,1.0,NO,Very Good,7 Average,1965,Seattle
4,510000.0,3,2.00,1680,8080,1.0,NO,Average,8 Good,1987,Sammamish


In [3]:
# Check for N/A values
homes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21597 entries, 0 to 21596
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   price        21597 non-null  float64
 1   bedrooms     21597 non-null  int64  
 2   bathrooms    21597 non-null  float64
 3   sqft_living  21597 non-null  int64  
 4   sqft_lot     21597 non-null  int64  
 5   floors       21597 non-null  float64
 6   waterfront   19221 non-null  object 
 7   condition    21597 non-null  object 
 8   grade        21597 non-null  object 
 9   yr_built     21597 non-null  int64  
 10  city         21597 non-null  object 
dtypes: float64(3), int64(4), object(4)
memory usage: 2.0+ MB


In [4]:
# Waterfront is the only column with N/A values,
homes.waterfront.value_counts()

NO     19075
YES      146
Name: waterfront, dtype: int64

In [5]:
# Replacing N/A with the mode
homes.waterfront.fillna('NO',inplace=True)

In [6]:
#Check basic statistics on the homes data
homes.describe()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,yr_built
count,2.159700e+04,21597.000000,21597.000000,21597.000000,2.159700e+04,21597.000000,21597.000000
mean,5.402966e+05,3.373200,2.115826,2080.321850,1.509941e+04,1.494096,1970.999676
std,3.673681e+05,0.926299,0.768984,918.106125,4.141264e+04,0.539683,29.375234
min,7.800000e+04,1.000000,0.500000,370.000000,5.200000e+02,1.000000,1900.000000
25%,3.220000e+05,3.000000,1.750000,1430.000000,5.040000e+03,1.000000,1951.000000
50%,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,1975.000000
75%,6.450000e+05,4.000000,2.500000,2550.000000,1.068500e+04,2.000000,1997.000000
max,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,2015.000000


In [7]:
# One home has 33 bedrooms and only 1620 sq_ft living
homes[homes.bedrooms > 8]

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,condition,grade,yr_built,city
4092,599999.0,9,4.50,3830,6988,2.5,NO,Average,7 Average,1938,Seattle
4231,700000.0,9,3.00,3680,4400,2.0,NO,Average,7 Average,1908,Seattle
6073,1280000.0,9,4.50,3650,5000,2.0,NO,Average,8 Good,1915,Seattle
8537,450000.0,9,7.50,4050,6504,2.0,NO,Average,7 Average,1996,Seattle
8748,520000.0,11,3.00,3000,4960,2.0,NO,Average,7 Average,1918,Seattle
13301,1150000.0,10,5.25,4590,10920,1.0,NO,Average,9 Better,2008,Bellevue
15147,650000.0,10,2.00,3610,11914,2.0,NO,Good,7 Average,1958,Bellevue
15856,640000.0,33,1.75,1620,6000,1.0,NO,Very Good,7 Average,1947,Seattle
16830,1400000.0,9,4.00,4620,5508,2.5,NO,Average,11 Excellent,1915,Seattle
18428,934000.0,9,3.00,2820,4480,2.0,NO,Average,7 Average,1918,Seattle


In [8]:
# Treating the above case as a data entry issue and replacing it with the median 3
homes.loc[homes.bedrooms > 20,'bedrooms'] = 3

In [9]:
# Checking the values for column with object dtype
homes.grade.value_counts()

7 Average        8974
8 Good           6065
9 Better         2615
6 Low Average    2038
10 Very Good     1134
11 Excellent      399
5 Fair            242
12 Luxury          89
4 Low              27
13 Mansion         13
3 Poor              1
Name: grade, dtype: int64

In [10]:
# There is an ordered numeric in the column so this extracts it
homes.grade = homes.grade.apply(lambda x: int(x.split()[0]))

In [11]:
# Replaced the grade with the numeric value from above
homes.grade.value_counts()

7     8974
8     6065
9     2615
6     2038
10    1134
11     399
5      242
12      89
4       27
13      13
3        1
Name: grade, dtype: int64

In [12]:
# Waterfront is binary so I encode 1 = Yes, 0 = No
Ord = OrdinalEncoder()
waterfronts = Ord.fit_transform(homes[['waterfront']])

# Default output is np array
waterfronts = pd.DataFrame(waterfronts,columns=['waterfront'])

# Remove the un-encoded column from dataset
homes = homes.drop('waterfront',axis=1)

In [13]:
# Checking the values for column with object dtype
homes.condition.value_counts()

Average      14020
Good          5677
Very Good     1701
Fair           170
Poor            29
Name: condition, dtype: int64

In [14]:
# Condition is categorical so I one hot encode
conditions = pd.get_dummies(homes.condition, prefix='cond',drop_first=True).astype(int)
conditions.columns = conditions.columns.str.replace(" ","")

# Remove the un-encoded column from dataset
homes = homes.drop('condition',axis=1)


In [15]:
# Checking the values for column with object dtype
homes.city.value_counts()

Seattle          8973
Renton           1597
Bellevue         1407
Kent             1201
Redmond           977
Kirkland          977
Auburn            911
Sammamish         800
Federal_Way       779
Issaquah          733
Maple_Valley      589
Woodinville       471
Snoqualmie        308
Kenmore           283
Mercer_Island     282
Enumclaw          233
North_Bend        220
Bothell           195
Duvall            190
Carnation         124
Vashon            117
Black_Diamond     100
Fall_City          80
Medina             50
Name: city, dtype: int64

In [16]:
# City is categorical so I one hot encode
cities = pd.get_dummies(homes.city,prefix='city',drop_first=True).astype(int)

# Remove the un-encoded column from dataset
homes = homes.drop('city',axis=1)

In [17]:
# All of the categoricals dropped and string columns changed to numeric
homes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21597 entries, 0 to 21596
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   price        21597 non-null  float64
 1   bedrooms     21597 non-null  int64  
 2   bathrooms    21597 non-null  float64
 3   sqft_living  21597 non-null  int64  
 4   sqft_lot     21597 non-null  int64  
 5   floors       21597 non-null  float64
 6   grade        21597 non-null  int64  
 7   yr_built     21597 non-null  int64  
dtypes: float64(3), int64(5)
memory usage: 1.5 MB


In [18]:
#pd.plotting.scatter_matrix(homes,figsize=(20,20));

In [19]:
#sns.heatmap(homes.corr())

In [20]:
# Re-add all of the categorical columns in their encoded forms
homes = pd.concat([homes,waterfronts,conditions,cities],axis=1)
homes = sm.add_constant(homes)

In [21]:
# Lots of encoded categoricals
homes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21597 entries, 0 to 21596
Data columns (total 37 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   const               21597 non-null  float64
 1   price               21597 non-null  float64
 2   bedrooms            21597 non-null  int64  
 3   bathrooms           21597 non-null  float64
 4   sqft_living         21597 non-null  int64  
 5   sqft_lot            21597 non-null  int64  
 6   floors              21597 non-null  float64
 7   grade               21597 non-null  int64  
 8   yr_built            21597 non-null  int64  
 9   waterfront          21597 non-null  float64
 10  cond_Fair           21597 non-null  int64  
 11  cond_Good           21597 non-null  int64  
 12  cond_Poor           21597 non-null  int64  
 13  cond_VeryGood       21597 non-null  int64  
 14  city_Bellevue       21597 non-null  int64  
 15  city_Black_Diamond  21597 non-null  int64  
 16  city

In [22]:
X = homes.drop('price',axis=1)
y = homes.price

X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=75,random_state=42)

In [23]:
# SKLearn model
sk_model = LinearRegression().fit(X_train,y_train)
print(sk_model.score(X_train,y_train))
print(sk_model.score(X_test,y_test))

0.8682783972638124
0.5866712367848487


In [24]:
# Statsmodels model
sm_model = sm.OLS(y_train,X_train).fit()
sm_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.868
Model:                            OLS   Adj. R-squared:                  0.797
Method:                 Least Squares   F-statistic:                     12.17
Date:                Thu, 17 Feb 2022   Prob (F-statistic):           1.84e-13
Time:                        16:08:14   Log-Likelihood:                -1001.6
No. Observations:                  75   AIC:                             2057.
Df Residuals:                      48   BIC:                             2120.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const               5.621e+06   2.21e+06      2.540      0.014    1.17e+06    1.01e+07
bedrooms           -5.615e+04   3.97e+04     -1.416      0.163   -1.36e+05    2.36e+04
bathrooms            6.11e+04   6.32e+04      0.967      0.339    -6.6e+04    1.88e+05
sqft_living           77.2603     60.850      1.270      0.210     -45.086     199.607
sqft_lot               0.9678      1.489      0.650      0.519      -2.026       3.962
floors              2478.3088   6.26e+04      0.040      0.969   -1.23e+05    1.28e+05
grade               1.712e+05   3.82e+04      4.485      0.000    9.44e+04    2.48e+05
yr_built           -3389.4936   1147.355     -2.954      0.005   -5696.405   -1082.582
waterfront          1.637e+06    2.4e+05      6.833      0.000    1.16e+06    2.12e+06
cond_Fair          -5.484e-10   6.23e-10     -0.880      0.383    -1.8e-09    7.05e-10
cond_Good           2.597e+04   6.98e+04      0.372      0.712   -1.14e+05    1.66e+05
cond_Poor           2.624e+05   3.17e+05      0.829      0.411   -3.74e+05    8.99e+05
cond_VeryGood           3e+04   8.07e+04      0.372      0.712   -1.32e+05    1.92e+05
city_Bellevue       4.414e+05   1.96e+05      2.247      0.029    4.64e+04    8.36e+05
city_Black_Diamond  1.968e-10   6.15e-11      3.202      0.002    7.32e-11     3.2e-10
city_Bothell       -5.655e-11   1.26e-10     -0.451      0.654   -3.09e-10    1.96e-10
city_Carnation      3.266e+05   2.57e+05      1.272      0.210    -1.9e+05    8.43e+05
city_Duvall         1.793e+05   2.59e+05      0.693      0.492   -3.41e+05       7e+05
city_Enumclaw      -7.473e-11   8.68e-11     -0.861      0.393   -2.49e-10    9.98e-11
city_Fall_City     -4.738e-11   4.46e-11     -1.062      0.293   -1.37e-10    4.23e-11
city_Federal_Way   -1.857e+05   1.91e+05     -0.970      0.337    -5.7e+05    1.99e+05
city_Issaquah       8.517e+04   2.58e+05      0.330      0.743   -4.34e+05    6.04e+05
city_Kenmore       -6.063e-11   4.64e-11     -1.308      0.197   -1.54e-10    3.26e-11
city_Kent           3946.4863   1.91e+05      0.021      0.984    -3.8e+05    3.88e+05
city_Kirkland       2.112e+05   2.02e+05      1.045      0.301   -1.95e+05    6.18e+05
city_Maple_Valley    1.64e+05   2.49e+05      0.659      0.513   -3.36e+05    6.64e+05
city_Medina        -6.771e-11   3.04e-11     -2.224      0.031   -1.29e-10   -6.48e-12
city_Mercer_Island  5.011e+05   2.61e+05      1.919      0.061   -2.39e+04    1.03e+06
city_North_Bend             0          0        nan        nan           0           0
city_Redmond        1.641e+05   1.88e+05      0.874      0.386   -2.13e+05    5.42e+05
city_Renton        -8.463e+04   2.59e+05     -0.327      0.745   -6.06e+05    4.36e+05
city_Sammamish      1.048e+05   2.15e+05      0.487      0.629   -3.28e+05    5.37e+05
city_Seattle        1.988e+05   1.57e+